In [ ]:
# main_csv_test.py

import time
import config
import pandas as pd 
from tqdm import tqdm 
from data_source.csv_reader import CSVReader
from pipeline.inference_pipeline import InferencePipeline

def main_test(csv_file_path: str):
    """
    Main function for testing the pipeline with a CSV file.
    Initializes the CSV reader and inference pipeline, then processes the file.
    """
    print("=" * 50)
    print("🚀 Starting CSI Inference Pipeline Test with CSV File.")
    print("=" * 50)

    try:
        # --- 1. Initialize pipeline and data source ---
        print("[1/3] Initializing Inference Pipeline...")
        pipeline = InferencePipeline(config)

        # 타임스탬프 컬럼 이름 설정
        timestamp_col = 'real_timestamp' 

        print(f"[2/3] Pre-calculating total chunks for progress bar...")
        # tqdm의 전체 진행률을 표시하기 위해 timestamp 컬럼만 먼저 읽어옴
        timestamps = pd.to_datetime(pd.read_csv(csv_file_path, usecols=[timestamp_col])[timestamp_col])
        total_duration_sec = (timestamps.iloc[-1] - timestamps.iloc[0]).total_seconds()

        # =======================================================
        # ✨ 디버깅 코드 추가 ✨
        # =======================================================
        print("-" * 20)
        print(f"🕒 First Timestamp: {timestamps.iloc[0]}")
        print(f"🕒 Last Timestamp:  {timestamps.iloc[-1]}")
        print(f"⏱️ Total Duration of CSV: {total_duration_sec:.2f} seconds")
        print(f"✂️ Window Size (from config): {config.WINDOW_SECONDS} seconds")
        print("-" * 20)
        # =======================================================

        total_chunks = int((total_duration_sec - config.WINDOW_SECONDS) / config.STEP_SECONDS) + 1

        # 만약 계산 결과가 음수이면 0으로 처리해주는 방어 코드 추가
        if total_chunks < 0:
            total_chunks = 0
            
        print(f"Total chunks to process: {total_chunks}")
        
        # 전체 청크 개수 계산
        total_chunks = int((total_duration_sec - config.WINDOW_SECONDS) / config.STEP_SECONDS) + 1
        print(f"Total chunks to process: {total_chunks}")

        print(f"[3/3] Initializing CSV Reader for file: {csv_file_path}...")
        csv_reader = CSVReader(
            file_path=csv_file_path,
            window_sec=config.WINDOW_SECONDS,
            step_sec=config.STEP_SECONDS,
            timestamp_col=timestamp_col 
        )

        print("\n✅ All components initialized successfully.")

    except Exception as e:
        print(f"❌ Critical error during initialization: {e}")
        return

    print("\n▶️ Starting inference process from CSV file...\n")
    start_time = time.time()
    
    # --- 2. Loop through data chunks from the CSV with tqdm ---
    # csv_reader를 tqdm으로 감싸고, 미리 계산한 total_chunks를 total 인자로 전달
    for csi_df_chunk in tqdm(csv_reader, total=total_chunks, desc="Processing Chunks"):
        if csi_df_chunk.empty:
            continue
        
        # --- 3. Execute pipeline ---
        result = pipeline.process(csi_df_chunk)

        # --- 4. Print results to terminal ---
        # tqdm 사용 시 일반 print는 줄바꿈을 유발하므로, tqdm.write를 사용하는 것이 깔끔합니다.
        if result:
            window_start = csi_df_chunk.index.min().strftime('%Y-%m-%d %H:%M:%S')
            tqdm.write(
                f"[{window_start}] Status: {result.get('status', 'N/A')}, "
                f"Movement: {result.get('movement', 'N/A')} (Conf: {result.get('movement_conf', 0.0):.2f}), "
                f"BPM: {result.get('bpm', 0.0):.2f}"
            )

    end_time = time.time()
    print("\n" + "=" * 50)
    print(f"✅ CSV processing finished.")
    print(f"Processed {total_chunks} chunks in {end_time - start_time:.2f} seconds.")
    print("=" * 50)


if __name__ == "__main__":
    # --- IMPORTANT ---
    # Change this path to your actual CSV file location
    CSV_FILE = "C:\\Users\\ssalt\\SOOM\\dataset\\full\\SLEEP_1010.csv"
    main_test(CSV_FILE)

In [5]:

import time
import config
import pandas as pd 
from tqdm import tqdm 
from data_source.csv_reader import CSVReader
from pipeline.inference_pipeline import InferencePipeline

In [3]:
csv_file_path = "C:\\Users\\ssalt\\SOOM\\dataset\\full\\SLEEP_1010.csv"
timestamp_col = 'real_timestamp'

In [10]:
timestamps = pd.read_csv(csv_file_path, usecols=[timestamp_col])[timestamp_col]
total_duration_sec = timestamps.iloc[-1] - timestamps.iloc[0]

# =======================================================
# ✨ 디버깅 코드 추가 ✨
# =======================================================
print("-" * 20)
print(f"🕒 First Timestamp: {timestamps.iloc[0]}")
print(f"🕒 Last Timestamp:  {timestamps.iloc[-1]}")
print(f"⏱️ Total Duration of CSV: {total_duration_sec:.2f} seconds")
print(f"✂️ Window Size (from config): {config.WINDOW_SECONDS} seconds")
print("-" * 20)
# =======================================================

total_chunks = int((total_duration_sec - config.WINDOW_SECONDS) / config.STEP_SECONDS) + 1

--------------------
🕒 First Timestamp: 15.025
🕒 Last Timestamp:  26260.686
⏱️ Total Duration of CSV: 26245.66 seconds
✂️ Window Size (from config): 4 seconds
--------------------


In [9]:
pd.read_csv(csv_file_path, usecols=[timestamp_col])

,real_timestamp
0,15.025
1,15.059
2,15.076
3,15.093
4,15.127
...,...
1304500,26260.601
1304501,26260.618
1304502,26260.635
1304503,26260.669
